In [1]:
# code by Tae Hwan Jung @graykode
import tensorflow as tf
import numpy as np
print(tf.__version__)
tf.reset_default_graph()

1.13.1


In [2]:
sentences = [ "i like dog", "i love coffee", "i hate milk"]

In [3]:
# ""中添加对划分之后的元素之间的符号,一个“”就是一个短语
word_list = "-- ".join(sentences)
word_list

'i like dog-- i love coffee-- i hate milk'

In [4]:
word_list = " ".join(sentences).split()
word_list

['i', 'like', 'dog', 'i', 'love', 'coffee', 'i', 'hate', 'milk']

In [5]:
# set 去重 之后 转为list
word_list = list(set(word_list))
word_list

['dog', 'i', 'milk', 'love', 'coffee', 'hate', 'like']

In [6]:
# enumerate枚举 
word_dict = {w: i for i, w in enumerate(word_list)}
word_dict

{'dog': 0, 'i': 1, 'milk': 2, 'love': 3, 'coffee': 4, 'hate': 5, 'like': 6}

In [7]:
number_dict = {i: w for i, w in enumerate(word_list)}
number_dict

{0: 'dog', 1: 'i', 2: 'milk', 3: 'love', 4: 'coffee', 5: 'hate', 6: 'like'}

In [8]:
n_class = len(word_dict) # number of Vocabulary
n_class

7

In [9]:
# NNLM Parameter
n_step = 2 # number of steps ['i like', 'i love', 'i hate']
n_hidden = 2 # number of hidden units

In [10]:
#  训练集是前两个词  标签是最后一个词 相当于one_hot
def make_batch(sentences):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]

        input_batch.append(np.eye(n_class)[input])
        target_batch.append(np.eye(n_class)[target])

    return input_batch, target_batch

In [11]:
test1 , test2 = make_batch(sentences)

In [12]:
test1

[array([[0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1.]]), array([[0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.]]), array([[0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0.]])]

In [13]:
test2,"\n",word_dict["dog"],"/n",np.eye(n_class)[0]   # dog 对应 0 、对应第一行

([array([1., 0., 0., 0., 0., 0., 0.]),
  array([0., 0., 0., 0., 1., 0., 0.]),
  array([0., 0., 1., 0., 0., 0., 0.])],
 '\n',
 0,
 '/n',
 array([1., 0., 0., 0., 0., 0., 0.]))

In [14]:
# Model
X = tf.placeholder(tf.float32, [None, n_step, n_class]) # [batch_size, number of steps, number of Vocabulary]
Y = tf.placeholder(tf.float32, [None, n_class])

input = tf.reshape(X, shape=[-1, n_step * n_class]) # [batch_size, n_step * n_class]
H = tf.Variable(tf.random_normal([n_step * n_class, n_hidden]))
d = tf.Variable(tf.random_normal([n_hidden]))
U = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

tanh = tf.nn.tanh(d + tf.matmul(input, H)) # [batch_size, n_hidden]
model = tf.matmul(tanh, U) + b # [batch_size, n_class]

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
prediction =tf.argmax(model, 1)  # 得到每行的最大值

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
# Training
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

input_batch, target_batch = make_batch(sentences)
print("input_batch ",input_batch,len(input_batch),"\n","target_batch", target_batch,len(target_batch))
for epoch in range(5000):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})
    if (epoch + 1)%1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

input_batch  [array([[0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.]]), array([[0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.]]), array([[0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.]])] 3 
 target_batch [array([1., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 1., 0., 0.]), array([0., 0., 1., 0., 0., 0., 0.])] 3
Epoch: 1000 cost = 0.418185
Epoch: 2000 cost = 0.062000
Epoch: 3000 cost = 0.024956
Epoch: 4000 cost = 0.012913
Epoch: 5000 cost = 0.007325


In [16]:
# Predict
predict =  sess.run([prediction], feed_dict={X: input_batch})
predict

[array([0, 4, 2], dtype=int64)]

In [17]:
number_dict[6],number_dict[2],number_dict[5]

('like', 'milk', 'hate')

In [18]:
# Test
input = [sen.split()[:2] for sen in sentences]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n] for n in predict[0]])

[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
